In [1]:
library(tidyverse)
library(dplyr)
library(repr)
library(tidymodels)
library(recipes)
install.packages("themis")
library(themis)
install.packages("GGally")
library(GGally)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

In [2]:
temp <- tempfile()
download_file <- download.file("https://archive.ics.uci.edu/static/public/186/wine+quality.zip", temp)
red_wine <- read.csv2(unz(temp, "winequality-red.csv")) |>
             mutate(type = "Red")
white_wine <- read.csv2(unz(temp, "winequality-white.csv")) |>
              mutate(type ="White")
head(red_wine)
head(white_wine)
unlink(temp)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,Red
2,7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,5,Red
3,7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,5,Red
4,11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,6,Red
5,7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,Red
6,7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,Red


,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6,White
2,6.3,0.3,0.34,1.6,0.049,14,132,0.994,3.3,0.49,9.5,6,White
3,8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,6,White
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6,White
5,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6,White
6,8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,6,White


In [6]:
wine_data <- rbind(red_wine, white_wine) |>
             mutate(across(fixed.acidity:alcohol, as.numeric),
                   type = as_factor(type))
write_csv(wine_data, "../data/wine_data.csv")
head(wine_data)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<fct>
1,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,Red
2,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,Red
3,7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,Red
4,11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,Red
5,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,Red
6,7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,Red


In [7]:
set.seed(2000)
#wine_data <- wine_data |>
#    select(fixed.acidity:alcohol, type)
wine_split <- initial_split(wine_data, prop = 0.75, strata = type)
wine_training <- training(wine_split)
wine_testing <- testing(wine_split)
head(wine_training)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<fct>
1,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,Red
2,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,Red
3,7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,Red
4,7.5,0.50,0.36,6.1,0.071,17,102,0.9978,3.35,0.80,10.5,5,Red
5,7.5,0.50,0.36,6.1,0.071,17,102,0.9978,3.35,0.80,10.5,5,Red
6,8.9,0.62,0.19,3.9,0.170,51,148,0.9986,3.17,0.93,9.2,5,Red


In [8]:
knn_recipe <- recipe(type ~ ., data = wine_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    prep()

wine_scaled_type <- bake(knn_recipe, wine_training)
head(wine_scaled_type)

#Then we do the upsampling
knn_recipe <- knn_recipe |>
 step_upsample(type, over_ratio = 1, skip = FALSE) |>
  prep()
knn_recipe

upsampled_wine_type <- bake(knn_recipe, wine_scaled_type)
# We check the upsampling result here
upsampled_wine_result <- upsampled_wine_type |>
  group_by(type) |>
  summarize(n = n())
upsampled_wine_result

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0.1379579,2.197977,-2.1840936,-0.7461269,0.5533783,-1.1054569,-1.4494401,1.0217267,1.8134883,0.2033448,-0.917050766,-0.9436766,Red
0.4465048,3.295474,-2.1840936,-0.6000828,1.1673877,-0.3079467,-0.8625687,0.6909617,-0.1114575,1.0153349,-0.581648282,-0.9436766,Red
0.1379579,1.954089,-2.1840936,-0.7669903,0.5254688,-0.9915268,-1.3427362,1.0217267,1.8134883,0.2033448,-0.917050766,-0.9436766,Red
0.2150946,0.978537,0.2787936,0.1301374,0.4138308,-0.7636668,-0.2401294,1.0217267,0.8199679,1.8273250,0.005306065,-0.9436766,Red
0.2150946,0.978537,0.2787936,0.1301374,0.4138308,-0.7636668,-0.2401294,1.0217267,0.8199679,1.8273250,0.005306065,-0.9436766,Red
1.2950087,1.710201,-0.8842365,-0.3288582,3.1768729,1.1731437,0.5779338,1.2863386,-0.2977426,2.7069810,-1.084752008,-0.9436766,Red




── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:    1
predictor: 12



── Training information 

Training data contained 4872 data points and no incomplete rows.



── Operations 

• Scaling for: fixed.acidity, volatile.acidity, citric.acid, ... | Trained

• Centering for: fixed.acidity, volatile.acidity, citric.acid, ... | Trained

• Up-sampling based on: type | Trained



type,n
<fct>,<int>
Red,3673
White,3673


In [11]:
mean_type <- upsampled_wine_type |>
    group_by(type) |>
    summarize(mean_fixed_acidity = mean(fixed.acidity),
                               mean_volatile_acidity = mean(volatile.acidity),
                               mean_citric_acid = mean(citric.acid),
                               mean_residual_sugar = mean(residual.sugar),
                               mean_chlorides = mean(chlorides),
                               mean_free_sufdioxide = mean(free.sulfur.dioxide),
                               mean_tot_sufdioxide = mean(total.sulfur.dioxide),
                               mean_density = mean(density),
                               mean_ph = mean(pH),
                               mean_sulphates = mean(sulphates),
                               mean_alcohol = mean(alcohol))
mean_type

type,mean_fixed_acidity,mean_volatile_acidity,mean_citric_acid,mean_residual_sugar,mean_chlorides,mean_free_sufdioxide,mean_tot_sufdioxide,mean_density,mean_ph,mean_sulphates,mean_alcohol
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Red,-4.887641,4.858869,-4.238361,-1.270926,23.648214,-1.778979,-2.075838,-104.4583,-16.43249,2.250498,-8.846971
White,-5.785619,-4.324927,-1.447445,-1.100599,-9.633544,-1.716679,-2.046949,-401.7610,-21.15329,-5.493718,-8.782726


Based on the results we got above, we are going to use mean_chlorides, mean_density to do the classification.

In [16]:
wine_training_class <- wine_training |>
    select(chlorides, density, type)
wine_testing_class <- wine_testing |>
    select(chlorides, density, type)
head(wine_training_class)
head(wine_testing_class)

,chlorides,density,type
,<dbl>,<dbl>,<fct>
1,0.076,0.9978,Red
2,0.098,0.9968,Red
3,0.075,0.9978,Red
4,0.071,0.9978,Red
5,0.071,0.9978,Red
6,0.170,0.9986,Red


,chlorides,density,type
,<dbl>,<dbl>,<fct>
1,0.092,0.9970,Red
2,0.075,0.9980,Red
3,0.076,0.9978,Red
4,0.069,0.9964,Red
5,0.065,0.9946,Red
6,0.073,0.9968,Red


In [17]:
knn_recipe <- recipe(type ~ ., data = wine_training_class)
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

knn_workflow <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_spec)

knn_res <- tune_grid(
    knn_workflow,
    resamples = vfold_cv(wine_training_class, v = 5),
    grid = data.frame(neighbors = seq(1, 50, 5))
)

best_k <- knn_res |>
    select_best("accuracy")

knn_final <- finalize_workflow(knn_workflow, best_k)

knn_fit <- fit(knn_final, data = wine_training_class)

knn_pred <- predict(knn_fit, new_data = wine_testing_class) |>
    bind_cols(wine_testing_class)

accuracy(knn_pred, truth = type, estimate = .pred_class)

knn_pred_num <- knn_pred |>
    mutate(type_num = as.numeric(type),
            .pred_class_num = as.numeric(.pred_class))

knn_rmse <- rmse(knn_pred_num, truth = type_num, estimate = .pred_class_num)

knn_rmse

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9286154


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,0.267179
